In [15]:
import re
from typing import Tuple
import numpy as np
from SolvingAdvent.HelferWichtel import HelferWichtel as Wichtel
from IPython.display import clear_output
import time
import math

TagFile = '11'


Helfer = Wichtel()
PuzzleInput = Helfer.getPuzzleInput(TagFile)
PuzzleInput = [line.replace('\n','').strip() for line in PuzzleInput]
print(PuzzleInput)

['Monkey 0:', 'Starting items: 72, 64, 51, 57, 93, 97, 68', 'Operation: new = old * 19', 'Test: divisible by 17', 'If true: throw to monkey 4', 'If false: throw to monkey 7', '', 'Monkey 1:', 'Starting items: 62', 'Operation: new = old * 11', 'Test: divisible by 3', 'If true: throw to monkey 3', 'If false: throw to monkey 2', '', 'Monkey 2:', 'Starting items: 57, 94, 69, 79, 72', 'Operation: new = old + 6', 'Test: divisible by 19', 'If true: throw to monkey 0', 'If false: throw to monkey 4', '', 'Monkey 3:', 'Starting items: 80, 64, 92, 93, 64, 56', 'Operation: new = old + 5', 'Test: divisible by 7', 'If true: throw to monkey 2', 'If false: throw to monkey 0', '', 'Monkey 4:', 'Starting items: 70, 88, 95, 99, 78, 72, 65, 94', 'Operation: new = old + 7', 'Test: divisible by 2', 'If true: throw to monkey 7', 'If false: throw to monkey 5', '', 'Monkey 5:', 'Starting items: 57, 95, 81, 61', 'Operation: new = old * old', 'Test: divisible by 5', 'If true: throw to monkey 1', 'If false: throw

In [16]:
class InputParser:
    def __init__(self):
        return
    
    def eatPuzzleInput(self, Input):
        '''Gibt jeden Monkey als gesamten String zurück'''
        inputString = ''.join(Input)
        a = re.finditer('Monkey \d+:',inputString)
        startPoints =[]
        for c in a:
            startPoints.append(c.start())
        startPoints.append(len(inputString))
        monkeyStrings=[]
        for e in range(0,len(startPoints)-1):
            monkeyStrings.append(inputString[startPoints[e]:startPoints[e+1]])
        return monkeyStrings
        
    def monkeyParser(self,monkeyString):
        indizesItems = re.search('Starting items:',monkeyString).span()
        indizesOperation = re.search('Operation:', monkeyString).span()
        indizesTest = re.search('Test:',monkeyString).span()
        items = monkeyString[indizesItems[1]: indizesOperation[0]]
        operation =monkeyString[indizesOperation[1]:indizesTest[0]]
        test = monkeyString[indizesTest[1]:]
        itemList = [int(item.strip()) for item in items.split(',')]
        operation = re.search(r'[+*] [\d]{0,3}[old]{0,6}',operation).group()
        digitsTest = re.findall('\d+',test)
        testDivisor = int(digitsTest[0])
        targetMonkeys = [int(digitsTest[1]), int(digitsTest[2])]
        return itemList, operation, testDivisor, targetMonkeys
    
    def createOperation(self, operand, value):
        return

class MonkeyGame:
    def __init__(self, listofMonkeys: dict[int,'Monkey'] = None):
        self.listOfMonkeys = listofMonkeys or {}
        self.inputParser = InputParser()
        
    def setupGame(self, PuzzleInput):
        monkeyStrings = self.inputParser.eatPuzzleInput(PuzzleInput)
        for n,monkeyString in enumerate(monkeyStrings):
            tokens = self.inputParser.monkeyParser(monkeyString)
            self.createMonkey(n,tokens[0],tokens[1],tokens[2],tokens[3])
        self.maximumWorry = math.prod([self.listOfMonkeys[monkey].testDivisor for monkey in self.listOfMonkeys])
        
    def createMonkey(self, monkeyNumber:int, startingItems:list[int], operation:str, testDivisor:int, targetMonkeys:list[int]):
        self.listOfMonkeys[monkeyNumber] = Monkey(self, startingItems, operation, testDivisor, targetMonkeys)
    
    def monkeyTurn(self, monkeyNumber):
        pass
    
    def gameRound(self):
        for monkey in self.listOfMonkeys:
            self.listOfMonkeys[monkey].inspectItems()
            
    def gameRoundPanic(self):
        for monkey in self.listOfMonkeys:
            self.listOfMonkeys[monkey].inspectItemsPanic(self.maximumWorry)
            
    def getListMonkeyInspections(self):
        listInspections =[]
        for monkey in self.listOfMonkeys:
            listInspections.append(self.listOfMonkeys[monkey].getNumberInspections())
        return listInspections
    
    def getMonkeyBusiness(self):
        a = sorted(self.getListMonkeyInspections())
        return a[-1]*a[-2]
    
    def monkeyThrow(self, receiver, item):
        monkey = self.listOfMonkeys[receiver]
        monkey.catchItem(item)
    
class Monkey:
    def __init__(self, game:'MonkeyGame', startingItems:list[int], operation:str, testDivisor:int, targetMonkeys:list[int]):
        self.items = startingItems
        self.numberInspections = 0
        self.ongoingGame = game
        self.operation = operation
        self.testDivisor = testDivisor
        self.targetMonkeys = targetMonkeys    
        
    def getNumberInspections(self):
        return self.numberInspections
    
    def executeOperation(self, item:int):
        if 'old' in self.operation:
            value = item
        else:
            value = int(re.search('\d+',self.operation).group())
        if self.operation[0] == '+':
            item += value
            return item
        if self.operation[0] == '*':
            item *= value
            return item
        else:
            raise Exception('Kein gültiger Operand')

    def executeTest(self, item:int):
        if not bool(item%self.testDivisor):
            return self.targetMonkeys[0]
        else:
            return self.targetMonkeys[1]
    
    def inspectItems(self):
        for item in self.items:
            item = self.executeOperation(item)
            item = item//3
            receiver = self.executeTest(item)
            self.throwItem(receiver, item)
            self.numberInspections +=1
        self.items =[]
        
    def inspectItemsPanic(self, maximumWorry):
        for item in self.items:
            item = self.executeOperation(item)
            item = item - (item//maximumWorry)*maximumWorry
            receiver = self.executeTest(item)
            self.throwItem(receiver, item)
            self.numberInspections +=1
        self.items =[]
    
    def throwItem(self, receiver:int, item:int):
        self.ongoingGame.monkeyThrow(receiver, item)
        
    def catchItem(self, item):
        self.items.append(item)

In [22]:
# Aufgabe A
newGame = MonkeyGame()
newGame.setupGame(PuzzleInput)
for i in range(20):
    newGame.gameRound()
print('Anzahl der geworfenen Items pro Affe:\n%s' %(sorted(newGame.getListMonkeyInspections())))
print('Ergebnis der Aufgabe A: %s' %newGame.getMonkeyBusiness())

Anzahl der geworfenen Items pro Affe:
[137, 160, 166, 172, 184, 218, 314, 318]
Ergebnis der Aufgabe A: 99852


In [23]:
# Aufgabe A
newGame = MonkeyGame()
newGame.setupGame(PuzzleInput)
for i in range(10000):
    newGame.gameRoundPanic()
print('Anzahl der geworfenen Items pro Affe:\n%s' %(sorted(newGame.getListMonkeyInspections())))
print('Ergebnis der Aufgabe B: %s' %newGame.getMonkeyBusiness())

Anzahl der geworfenen Items pro Affe:
[25869, 27041, 30343, 146733, 155504, 160360, 160567, 161523]
Ergebnis der Aufgabe B: 25935263541


In [92]:
# Class Test
newGame = MonkeyGame()
newGame.createMonkey(0,[10,20,11],'* 10', 13, [1,2])
newGame.createMonkey(1,[10,20,11],'* 10', 13, [2,0])
newGame.createMonkey(2,[10,20,11],'* 10', 13, [0,1])
newGame.gameRound()

In [44]:
a = PuzzleInput[2]
re.findall(r'[+*] \d+',a)

['* 19']

In [171]:
inputString = ''.join(PuzzleInput)
b = re.finditer('Monkey \d+:',inputString)
startPoints =[]
for c in b:
    startPoints.append(c.start())
startPoints.append(len(inputString))
monkeyStrings=[]
for e in range(0,len(startPoints)-1):
    monkeyStrings.append(inputString[startPoints[e]:startPoints[e+1]])
print('\n'.join(monkeyStrings))

for monkeyString in monkeyStrings:
    indizesItems = re.search('Starting items:',monkeyString).span()
    indizesOperation = re.search('Operation:', monkeyString).span()
    indizesTest = re.search('Test:',monkeyString).span()
    items = monkeyString[indizesItems[1]: indizesOperation[0]]
    operation =monkeyString[indizesOperation[1]:indizesTest[0]]
    test = monkeyString[indizesTest[1]:]
    itemList = [int(item.strip()) for item in items.split(',')]
    operation = re.search(r'[+*] [\d+]?[old]{0,6}',operation).group()
    digitsTest = re.findall('\d+',test)
    testDivisor = int(digitsTest[0])
    targetMonkeys = [int(digitsTest[1]), int(digitsTest[2])]
# startIndizes

Monkey 0:Starting items: 72, 64, 51, 57, 93, 97, 68Operation: new = old * 19Test: divisible by 17If true: throw to monkey 4If false: throw to monkey 7
Monkey 1:Starting items: 62Operation: new = old * 11Test: divisible by 3If true: throw to monkey 3If false: throw to monkey 2
Monkey 2:Starting items: 57, 94, 69, 79, 72Operation: new = old + 6Test: divisible by 19If true: throw to monkey 0If false: throw to monkey 4
Monkey 3:Starting items: 80, 64, 92, 93, 64, 56Operation: new = old + 5Test: divisible by 7If true: throw to monkey 2If false: throw to monkey 0
Monkey 4:Starting items: 70, 88, 95, 99, 78, 72, 65, 94Operation: new = old + 7Test: divisible by 2If true: throw to monkey 7If false: throw to monkey 5
Monkey 5:Starting items: 57, 95, 81, 61Operation: new = old * oldTest: divisible by 5If true: throw to monkey 1If false: throw to monkey 6
Monkey 6:Starting items: 79, 99Operation: new = old + 2Test: divisible by 11If true: throw to monkey 3If false: throw to monkey 1
Monkey 7:Start

In [144]:
a = 'abcdefg'
a.s
print(a.split(['b']))

TypeError: must be str or None, not list

In [58]:
b = {}
c = {0:'Null'}
print(c)
c[1] = 'Eins'
print(c)
print(c[0])



{0: 'Null'}
{0: 'Null', 1: 'Eins'}
Null


In [64]:
a = [1,2,3]
print(a)
a[1] = 5
print(a)

[1, 2, 3]
[1, 5, 3]


In [70]:
c = {}
c[1] = MonkeyGame()
print(c)

{1: <__main__.MonkeyGame object at 0x000001375C86A0E0>}


In [172]:
'old' in 'asdsdold'

True